# Voluntary REDD Analysis (Google Earth Engine)
---

Computes change in deforestation for Brazilian municipalities using annual land cover classifications from [MapBiomas](https://mapbiomas.org/) (1985 - 2018).

Based on Voluntary REDD Analysis developed by Thales West.
[Github](https://github.com/thaleswest/Voluntary-REDD-analysis)

To execute the code, you must have an account with Google Earth Engine. If you do not have an account, register and sign-up with Earth Engine at https://earthengine.google.com/.

---
Developed by Kyle A. Jones

San Diego State University, Department of Geography

kjones1697@sdsu.edu

### Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro).

If you need to visualize intermediate or final results, uncomment install of `geehydro`. The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.

***Uncomment these lines if you are running this notebook for the first time.***

In [ ]:
# !pip install earthengine-api
# !pip install geehydro

###Import libraries

Uncomment lines to import foluim and geehydro if needing to visualize output.

In [ ]:
import ee
# import folium
# import geehydro

###Authenticate and initialize Earth Engine API.

You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for the first time or if you are getting an authentication error.  

In [ ]:
ee.Authenticate()
ee.Initialize()

### Define study years and import datasets.
Input: 

1.   Start and End Study Years
2.   MapBiomas `collection41_intergration_v1`
3.   Municipal boundaries (**2015** subset of municipalities)

[MapBiomas Classes](https://storage.googleapis.com/mapbiomas/mapbiomas-br/v4/download%20_%20codigo%20legenda/C%C3%B3digos%20da%20legenda%20Cole%C3%A7%C3%A3o%204.pdf) (Collection 4.1)

In [ ]:
## Define Study Years (Start - End)
startDate = 1985
endDate = 2018

num_years = (endDate-startDate+1)

## Load data
mb_landcover = ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
municipalities = ee.FeatureCollection('projects/Biggs-Lab/MUNIC_2015')

## Uncomment IF performing analysis on all municipalities in the original 'municipalities' feature Collection.
municip_table = municipalities

# ## Uncomment IF performing analysis on municipalities in a specific Brazilian state, enter the corresponding state code.
# ## This value corresponds to the first two digits for a given municipality (i.e. 11 = Rondonia)
# ## Define State Code
# state_code = 12

# ## Filter municipalities layer by municipalities in a given state.
# municip_table = municipalities.filterMetadata('STATE_CODE', 'equals', state_code)

### Define Functions

In [ ]:
## Step 2
def reclass_img_to_col(yr):
  """Function 'reclass_img_to_col' creates one annual image for each study year using the original
  MapBiomas landcover image. Pixel values in each annual image are reclassified from the MapBiomas
  categories to new landcover categories: forest, non-forest, water, and cloud/NA."""
  year = ee.Number(yr)
  class_yr = ee.String('classification_').cat(year.format("%d"))
  img = mb_landcover.select(class_yr)
  img = img.remap(mapbiomas_lc, new_lc)
  img = img.rename(class_yr)                    # image bands are labeled 'remapped' following .remap(). Change back to 'classifcation_YYYY'
  img = img.set('YearID', year.format("%d"))    # Add new metadata property 'YearID' to be used in filtering collection or identify images by year
  img = img.set('date', year)                   # Add new metadata property 'date' to be used in filtering collection or identify images by year
  return img

## Step 3
def replace_cloud(current, previous):
  """Function 'replace_cloud' checks if pixel in current year is cloud/NA (0). If true, the
  current pixel value is updated to the pixel value of previous year."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(current.eq(0), last_image)
  return ee.List(previous).add(updated)

## Step 4
def mask_cloud(image):
  """Function 'mask_cloud' updates pixel values in all 'cloud_col' images
  based on cloud/NA (0) pixels in cloud_mask"""
  updated = image.where(cloud_mask.eq(1), 0)
  return updated

## Step 5
def change_water(current, previous):
  """Function 'change_water' checks if pixel in previous year is water (3) and same pixel
  in current year is not water. If true, the current pixel value is updated to water (3)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(3).And(current.neq(3)), 3)
  return ee.List(previous).add(updated)

## Step 6
def mask_water(image):
  """Function 'mask_water' is used to update pixel values in all 'water_col' images based on
  water (3) pixels in 'water_mask' making the water pixels consistent in all study years."""
  updated = image.where(water_mask.eq(1), 3)
  return updated

## Step 7
def to_nonforest(current, previous):
  """Funtion 'to_nonforest' flags forest to non-forest transition. If pixel is
  forest (1) in previous year and non-forest (2) in current year, flag current pixel (4)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(1).And(current.eq(2)), 4)
  return ee.List(previous).add(updated)

def repeat_nonforest(current, previous):
  """Funtion 'repeat_nonforest' checks if non-forest transition is stable in two sequential years. If pixel
  is flagged (4) in previous year and non-forest (2) in current year, flag current pixel (4)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(4).And(current.eq(2)), 4)
  return ee.List(previous).add(updated)

## Update 1 - correct from F-N-N-F to F-N-F-F
def return_nonfor_to_forest(image):
  """Function 'return_nonfor_to_forest' compares the current image to the next image. If the
  current image is flagged (4) and next image is forest (1), flagged pixels are changed back to
  forest."""
  img_date = ee.Number(image.get('date'))
  next = ee.Algorithms.If(img_date.lt(endDate),             # If image date is the final study year - no 'next' image.
                          ee.Image(flag_col.filterMetadata('date','equals', img_date.add(1)).first()),
                          image)
  updated = image.where(image.eq(4).And(ee.Image(next).eq(1)), 1)
  return updated

## Update 2 - correct from F-N-F-F to F-F-F-F
def repeat_return_nonfor_to_forest(image):
  """Function 'repeat_return_nonfor_to_forest' compares the current image to the next image. If the
  current image is flagged (4) and next image is forest (1), flagged pixels are changed back to
  forest. Relassify all remaining flagged pixels (.remap()) back to non-forest (2)"""
  img_date = ee.Number(image.get('date'))
  next = ee.Algorithms.If(img_date.lt(endDate),             # If image date is the final study year - no 'next' image.
                          ee.Image(fnff_updated_col.filterMetadata('date','equals', img_date.add(1)).first()),
                          image)
  updated = image.where(image.eq(4).And(ee.Image(next).eq(1)), 1)
  updated = updated.remap([0,1,2,3,4], [0,1,2,3,2])         # Remap remaining flagged (4) pixels back to non-forest (2)
  updated = updated.rename('Landcover')                     # Rename to image band name from 'remapped' to 'Landcover'
  return updated

## Step 8
def fix_forest(current, previous):
  """Function 'fix_forest' fixes non-forest to forest transition. If pixel in previous year is non-forest (2)
  and same pixel in current year is forest (1), update current pixel back to non-forest (2)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(2).And(current.eq(1)), 2)
  return ee.List(previous).add(updated)

## Step 9
def get_area(image):
    """Function 'get_area' computes the area of forest pixels in each image outside of
    reduce_region function so the process does not need to be repeated each time a new
    feature is used within 'reduceRegion'."""
    area = image.eq(1).multiply(ee.Image.pixelArea().divide(1000000))
    return area.copyProperties(image)

## Option 1 - Use reduceRegions() and map() over Image Collection
def reduce_regions(image):
  """Function 'reduce_regions' uses reduceRegions() with reducer: sum() to complute the
  total area (km) of forested pixels in each municipality feature"""
  reduced = image.reduceRegions(collection = municip_table, reducer = ee.Reducer.sum(), scale = 30)
  reduced = reduced.filter(ee.Filter.neq('sum', None))
  reduced = reduced.map(lambda f: f.set('date', image.get('YearID')))
  return reduced

## Option 2 - Use reduceRegion() and map over Feature Collection
def reduce_region(feature):
  """Function maps over each municipal feature and then each annual image using reduceRegion() with
  reducer: sum() to complute the total number of forested pixels in each municipality"""
  def by_image(image):
    reduced = ee.Feature(None, image.reduceRegion(reducer = ee.Reducer.sum(), geometry = feature.geometry(), scale = 30, maxPixels = 1e10))
    reduced = reduced.set('date', image.get('YearID')).copyProperties(feature)
    return reduced
  per_feat = final_col_area.map(by_image)
  return per_feat.copyProperties(feature)

## Table Formatting (Triplets)
## IMPORTANT: Update feature.get() in function col_id_val to 'sum' if mapping over image collection (Option 1)
## and 'Landcover' if mapping over feature collection (Option 2).
def format(table, rowID, colID):
  """Function 'format' defines a 2D table of rowId x colId from a table of triplets."""
  ## Get a FeatureCollection with unique row IDs.
  rows = table.distinct(rowID)
  ## Join the table to the unique IDs to get a collection in which each feature stores a list of all features having a common row ID.
  joined = ee.Join.saveAll('matches').apply(
      primary = rows,
      secondary = table,
      condition = ee.Filter.equals(
          leftField = rowID,
          rightField = rowID
      )
  )
  def unique_id(row):
    values = ee.List(row.get('matches'))
    def col_id_val(feat):
      feature = ee.Feature(feat)
      id_val = [feature.get(colID), feature.get('Landcover')]   ## CHANGE HERE
      return id_val
    values = values.map(col_id_val)
    return row.select([rowID]).set(ee.Dictionary(values.flatten())) 
  return joined.map(unique_id)



### Reclassify the original MapBiomas land cover classes for years 1985 - 2018

In [ ]:
## Step 2. Reclassify the original MapBiomas land cover classes for years 1985 - 2018
## Input: mb_landcover from Step 1

## 1 = New/Reclassified forest - Based on original pixel values = 1, 2, 3, 4, and 5 (Do not include "forest plantation" [9] and "non forest natural formation" [10])
## 2 = New/Reclassified non-forest - Based on original pixel values = 9 to 15, 18 to 25, 29 to 32
## 3 = New/Reclassified water - Based on original pixel values = 26 and 33
## 0 = New/Reclassified cloud cover/NA - Based on the original "non observed" class (pixel value = 27)

## Lists of original MapBiomas landcover (lc) classes and NEW landcover classes
mapbiomas_lc = [1,2,3,4,5,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25,26,27,29,30,31,32,33]
new_lc = [1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,0,2,2,2,2,3]

## Create list of study years to map over
years_list = ee.List.sequence(startDate, endDate)

## Create image collection from band names
image_col = ee.ImageCollection.fromImages(ee.List(years_list.map(reclass_img_to_col)))


###------###
## Step 3. Replace Cloud/NA pixels (start year to last mapped year)
## Input: image_col from Step 2.

## Important: image_col is REVERSED so the process of updating pixels goes from endDate to startDate.
## If pixel in current image has a value of "0," get pixel value at same location in following year. The
## corrected image is then used in the next iteration (i.e. 2018 >> 2017, 2017 >> 2016, ...). After
## applying function replace_cloud, REVERSE the resulting list, cloud_col, back to the original order
## (i.e. startDate to endDate) for next steps.

## Apply function 'replace_cloud'.
## Reverse the input image collection by year (i.e. descending 2018 - 1985)
image_col_desc = image_col.sort('date',False)

## Select first image in collection (last study year) as initial image to pass into iterate.
image_initial = ee.Image(image_col_desc.first())

## Use remaining images in collection (excluding the initial image) to iterate over.
images_to_iterate = image_col_desc.filterMetadata('date','less_than',endDate)

## Iterate over collection and apply replace_cloud function. Output as new image collection, REVERSING
## the order of the image collection back to original (i.e. 1985 to 2018)
cloud_col = ee.ImageCollection(ee.List(images_to_iterate.iterate(replace_cloud, ee.List([image_initial])))).sort('date')


###------###
## Step 4. Apply cloud mask based on the second-to-last mapped year
## Input: cloud_col from Step 3 and a new layer, cloud_mask, created from the second-to-last cloud_col image.

## Function mask_cloud is used to update pixel values in all cloud_col images based on cloud/NA (0) pixels in
## cloud_mask to mitgate bias from cloud/NA pixels in final results. The updated images are output as
## cloud_mask_col for use in Step 5. If 2018 is the last mapped year, 2017 is used for cloud mask so the opportunity
## to replace NA/cloud (0) pixels in 2017 with the land use observed in 2018 (assuming those pixels are not cloud in 2018) exists.

## Create the cloud mask. Creates binary image (1 = cloud; 0 = no cloud).
cloud_mask = ee.Image(cloud_col.filterMetadata('date','equals', (endDate-1)).first()).eq(0)

## Apply function 'mask_cloud'
cloud_mask_col = cloud_col.map(mask_cloud)


###------###
## Step 5. Fix water transitions (1985 to last mapped year)
## Input: cloud_mask_col from Step 4

## Uses sequential annual classifications to identify temporally prohibited transitions. Results
## from the previous year are compared to the current year.

## Apply function 'change_water'.
## Select first image in 'cloud_mask_col' collection as initial image to pass into iterate.
cloud_initial = ee.Image(cloud_mask_col.first())

## Use remaining images in 'cloud_mask_col' (excluding the initial image) to iterate over.
cloud_to_iterate = cloud_mask_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply change_water function. Output as image collection.
water_col = ee.ImageCollection(ee.List(cloud_to_iterate.iterate(change_water, ee.List([cloud_initial]))))


###------###
## Step 6. Apply water mask based on the last mapped year.
## Input: water_col from Step 5 and a new layer, water_mask, created from the last water_col image.

## Function mask_water is used to update pixel values in all water_col images based on water (3) pixels
## in water_mask making the water pixels consistent in all study years. The results are added to water_mask_col for use in Step 7.

## Create the water mask. Creates binary image (1 = water; 0 = no water).
water_mask = ee.Image(water_col.filterMetadata('date','equals', endDate).first()).eq(3)

## Apply function 'mask_water'
water_mask_col = water_col.map(mask_water)


###------###
## Step 7. Identify and fix prohibited forest - non-forest - forest transitions.
## Input: water_mask_col from Step 6.

## Both functions use results from the previous image (backward comparison) and require .interate(). The first function
## flags (4) pixels undergoing the forest (1) to non-forest (2) transition. The second function checks to see if flagged (4)
## pixels remain non-forest (2) in the following year. If so, the non-forest (2) pixels are also flagged (4).
## 4 is the value used to flag pixels.

## Apply function 'to-nonforest'.
## Select first image in 'water_mask_col' collection as initial image to pass into iterate.
water_initial = ee.Image(water_mask_col.first())

## Use remaining images in 'water_mask_col' (excluding the initial image) to iterate over.
water_to_iterate = water_mask_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply change_water function. Output as image collection.
nonforest_col = ee.ImageCollection(ee.List(water_to_iterate.iterate(to_nonforest, ee.List([water_initial]))))


## Apply function 'repeat_nonforest'.
## Select first image in 'nonforest_col' collection as initial image to pass into iterate.
nonforest_initial = ee.Image(nonforest_col.first())

## Use remaining images in 'nonforest_col' (excluding the initial image) to iterate over.
nonforest_to_iterate = nonforest_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply 'repeat_nonforest' function. Output as image collection.
flag_col = ee.ImageCollection(ee.List(nonforest_to_iterate.iterate(repeat_nonforest, ee.List([nonforest_initial]))))

###----###
## With "forest - nonforest" and "forest - nonforest - nonforest" pixels flagged, compare the current image to
## the next image (forward comparison). If pixels are flagged (4) in Image n and forest (1) in Image n+1, update the
## flagged pixels Image n to forest (1). The process to update pixel values is repeated twice to fix consecutive years
## of flagged pixels (i.e. F-N-N-F to F-N-F-F and then F-N-F-F to F-F-F-F):

## 1) forest to non-forest (flagged) to forest
## 2) forest to non-forest (flagged) to non-forest (flagged) to forest

## Update 1) map the return_nonfor_to_forest function over flag_col image collection.

## Update 2) map the repeate_return_nonfor_to_forest function over fnff_updated_colimage collection. After the final
## updates, relassify all remaining flagged pixels (.remap()) back to non-forest (2).


## Update 1 - correct from F-N-N-F to F-N-F-F
## Apply function 'return_nonfor_to_forest'
fnff_updated_col = flag_col.map(return_nonfor_to_forest)

## Update 2 - correct from F-N-F-F to F-F-F-F
## Apply function 'repeat_return_nonfor_to_forest'
ffff_updated_col = fnff_updated_col.map(repeat_return_nonfor_to_forest)


###------###
## Step 8. Fix logical errors in non-forest - forest transition.
## Input: ffff_updated_col from Step 7.

## The fix_forest function identifies non-forest (2) to forest (1) transition pixels. It changes forest (1)
## pixels back to non-forest (2) to avoid false reforestation that will result in overestimation of forest.

## Apply function 'fix_forest'.
## Select first image in 'ffff_col' collection as initial image to pass into iterate.
ffff_initial = ee.Image(ffff_updated_col.first())

## Use remaining images in 'ffff_col' (excluding the initial image) to iterate over.
ffff_to_iterate = ffff_updated_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply fix_forest function. Output as final image collection.
final_collection = ee.ImageCollection(ee.List(ffff_to_iterate.iterate(fix_forest, ee.List([ffff_initial]))))    # FINAL IMAGE COLLECTION


###------###
## Step 9. Compute deforestation over time by municipalities.
## Input: 'final_collection'

## Apply function 'get_area'.
final_col_area = final_collection.map(get_area)


## TWO OPTIONS TO COMPUTE # of Forest Pixels in each municipality.
## 1) Use reduceRegions() and map() over Image Collection
## 2) Use reduceRegion() and map() over Feature Collection

# ## Option 1 - Use reduceRegions() and map() over Image Collection           ## COMMENT ON/OFF DEPENDING ON METHOD
# ## Apply to Image Collection
# regions_result = final_col_area.map(reduce_regions)
# regions_result = regions_result.flatten()
# ## Remove contents of '.geo' for clarity
# results_out = regions_result.select(['.*'], None, False)


## Option 2 - Use reduceRegion() and map over Feature Collection          ## COMMENT ON/OFF DEPENDING ON METHOD
# Apply to Feature Collection
region_result = municip_table.map(reduce_region)
results_out = region_result.flatten()


###------###
## Step 10. Format a table of triplets to organize output.
## IMPORTANT: Update feature.get() in function col_id_val in FUNCTION "format" above to 'sum' if mapping over image collection
## and 'Landcover' if mapping over feature collection.

## Adjust format of table coloumns and rows
output_table = format(results_out, 'CD_GEOCMU', 'date');

### Export Results

In [ ]:
## Set export properites
properties = {'driveFolder': 'RFF',  'driveFileNamePrefix': 'OPT2_All_Municip'+'_Y'+str(startDate)+'_'+str(endDate), 'fileFormat': 'CSV'}

## Define the task in Python
task = ee.batch.Export.table(output_table, 'OPT2_All_Municip'+'_Y'+str(startDate)+'_'+str(endDate), properties)


# ## Uncomment if running for specific state (state code will populate file name)
# ## Set export properites
# properties = {'driveFolder': 'RFF',  'driveFileNamePrefix': 'OPT2_S'+str(state_code)+'_Y'+str(startDate)+'_'+str(endDate), 'fileFormat': 'CSV'}

# ## Define the task in Python
# task = ee.batch.Export.table(output_table, 'OPT2_S'+str(state_code)+'_Y'+str(startDate)+'_'+str(endDate), properties)


## Start the task, equivalent to hit the "run" button in the editor
task.start()

### Create an interactive map to visualize results (Optional)
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [1]:
## Set 'test' equal to desired image for visualization.
test = ee.Image(final_result.get(3))

## Set starting location to center on Rhondonia
Map = folium.Map(location=[-11, -63], zoom_start=6)
Map.setOptions('ROADMAP')

## Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4,
  'palette': ['000000', '76B37F', 'F0E5AA', '03A5FC', 'FC03DB']}  #[Black, Olive, Beige, Blue, Fuchsia]

## Add selected layer to map.
Map.addLayer(test, vis_params, 'Landcover')
Map.addLayer(select_mun)
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map